In [ ]:
# default_exp core.database_rq

In [ ]:
#hide
%reload_ext autoreload
%autoreload 2

In [ ]:
#hide
from nbdev.showdoc import *
import json
import responses
from pathlib import Path
from discogspy.core import *

In [ ]:
#hide
with open(Path("../config/example_user_info.json"), "r") as user_config_file:
    config = json.load(user_config_file)
    example_user = UserWithUserTokenBasedAuthentication(user_token=config["user_token"],
                                                        user_agent=config["user_agent"])

# Core Database Requests

> function wrapper around the discogs api for database interactions

In [ ]:
#export
import requests
from typing import Union
from discogspy.core import *

## Release

In [ ]:
#export


def get_release(user: Union[UserWithoutAuthentication,
                            UserWithUserTokenBasedAuthentication], 
                release_id: int, 
                curr_abbr: Union[str, None]) -> requests.models.Response:
    """
    Get information to a particular release from discogs database.
    A release represents a particular physical or digital object released by 
    one or more Artists.
    
    No user Authentication needed.
    """
    url = f"{RELEASES_URL}/{release_id}"
    headers = user.headers
    params = user.params
    if curr_abbr and curr_abbr in VALID_CURR_ABBR: 
        params["curr_addr"] = curr_abbr
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_releases
get_release(user=example_user, release_id=6067651, curr_abbr="EUR")

<Response [200]>

In [ ]:
#hide 

@responses.activate
def test_get_release____authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{RELEASES_URL}/123456?curr_addr=EUR"
    result_content = {"TestRelease": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release(user=test_user, release_id=123456, curr_abbr="EUR")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release____authentication_none()

@responses.activate
def test_get_release___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456?token=test_user_token&curr_addr=EUR"
    result_content = {"TestRelease": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release(user=test_user, release_id=123456, curr_abbr="EUR")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release___authentication_user_token()

### Release Rating

In [ ]:
#export


def get_release_rating_by_user(user: Union[UserWithoutAuthentication,
                                           UserWithUserTokenBasedAuthentication], 
                               release_id: int, 
                               username: str) -> requests.models.Response:
    """
    Get the rating of a release made by a given user.
    
    No user Authentication needed. 
    """
    url = f"{RELEASES_URL}/{release_id}/rating/{username}"
    headers = user.headers
    params = user.params
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_release_rating_by_user
get_release_rating_by_user(user=example_user, release_id=9453891, username="darkwolf")


<Response [200]>

In [ ]:
#hide 

@responses.activate
def test_get_release_rating_by_user____authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{RELEASES_URL}/123456/rating/test_user_name"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release_rating_by_user(user=test_user, release_id=123456, username="test_user_name")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release_rating_by_user____authentication_none()

@responses.activate
def test_get_release_rating_by_user___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release_rating_by_user(user=test_user, release_id=123456, username="test_user_name")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release_rating_by_user___authentication_user_token()

In [ ]:
# export


def update_release_rating_for_given_user(user: UserWithUserTokenBasedAuthentication, 
                                         release_id: int, 
                                         username: str,
                                         rating: int) -> requests.models.Response:
    """
    Update the rating of a release made by a given user.
    If there is no rating, it will create one.
    
    User Authentication needed. 
    """
    url = f"{RELEASES_URL}/{release_id}/rating/{username}"
    headers = user.headers
    params = user.params
    rating = min(max(0, rating), 5)
    data = {"rating": rating}
    return requests.put(url, headers=headers, params=params, json=data)

In [ ]:
# Example for usage of update_release_rating_for_given_user

update_release_rating_for_given_user(example_user, release_id=11690093, 
                                     username="derneueweg-label", rating=5)

<Response [201]>

In [ ]:
#hide
@responses.activate
def update_release_rating_for_given_user___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.PUT, target_url,
                  json=result_content, status=201)

    resp = update_release_rating_for_given_user(test_user, release_id=123456, 
                                                username="test_user_name", rating=5)
   
    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

update_release_rating_for_given_user___authentication_user_token()

@responses.activate
def update_release_rating_for_given_user_with_rating_higher_5___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.PUT, target_url,
                  json=result_content, status=201)

    resp = update_release_rating_for_given_user(test_user, release_id=123456, 
                                                username="test_user_name", rating=6)
   
    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

update_release_rating_for_given_user_with_rating_higher_5___authentication_user_token()

@responses.activate
def update_release_rating_for_given_user_with_rating_lower_0___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {"username": "test_user_name", "rating": 5, "release_id": 123456}
    
    responses.add(responses.PUT, target_url,
                  json=result_content, status=201)

    resp = update_release_rating_for_given_user(test_user, release_id=123456, 
                                                username="test_user_name", rating=-5)
   
    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

update_release_rating_for_given_user_with_rating_lower_0___authentication_user_token()

In [ ]:
# export


def delete_release_rating_by_user(user: UserWithUserTokenBasedAuthentication, 
                                  release_id: int, 
                                  username: str) -> requests.models.Response:
    """
    Delete the rating of a release made by a given user.
    
    User Authentication needed. 
    """
    url = f"{RELEASES_URL}/{release_id}/rating/{username}"
    headers = user.headers
    params = user.params
    return requests.delete(url, headers=headers, params=params)

In [ ]:
# Example for usage of delete_release_rating_by_user

delete_release_rating_by_user(example_user, release_id=11690093, username="derneueweg-label")

<Response [204]>

In [ ]:
#hide

@responses.activate
def test_delete_release_rating_by_user___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{RELEASES_URL}/123456/rating/test_user_name?token=test_user_token"
    result_content = {}
    
    responses.add(responses.DELETE, target_url,
                  json=result_content, status=204)

    resp = delete_release_rating_by_user(user=test_user, release_id=123456, username="test_user_name")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_delete_release_rating_by_user___authentication_user_token()

In [ ]:
# export


def get_release_rating_by_community(user: Union[UserWithoutAuthentication,
                                                UserWithUserTokenBasedAuthentication], 
                                    release_id: int) -> requests.models.Response:
    """
    Get the rating of a release made by the community.
    A community release rating includes the average rating and 
    the total number of user ratings for a given release.
    
    This function doesn't work for master releases!
    
    No user Authentication needed. 
    """
    url = f"{RELEASES_URL}/{release_id}/rating"
    headers = user.headers
    params = user.params
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of delete_release_rating_by_user

get_release_rating_by_community(example_user, release_id=1995816)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_release_rating_by_community___authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{RELEASES_URL}/123456/rating"
    result_content = {"rating": {"count": 19, "average": 4.89}, "release_id": 1995816}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release_rating_by_community(user=test_user, release_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release_rating_by_community___authentication_none()

@responses.activate
def test_get_release_rating_by_community___authentication_user_token():

    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{RELEASES_URL}/123456/rating?token=test_user_token"
    result_content = {"rating": {"count": 19, "average": 4.89}, "release_id": 1995816}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_release_rating_by_community(user=test_user, release_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_release_rating_by_community___authentication_user_token()

### Master Release

In [ ]:
#export


def get_master_release(user: Union[UserWithoutAuthentication,
                                   UserWithUserTokenBasedAuthentication], 
                       master_id: int) -> requests.models.Response:
    """
    Get information to a particular master release from discogs database.
    A Master release represents a set of similar Releases. 
    Masters releases have a “main release” which is often the chronologically earliest.
    
    No user Authentication needed.
    """
    url = f"{MASTERS_URL}/{master_id}"
    headers = user.headers
    params = user.params
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_master_release

get_master_release(example_user, master_id=195663)

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_master_release___authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{MASTERS_URL}/123456"
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_master_release(user=test_user, master_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_master_release___authentication_none()

@responses.activate
def test_get_master_release___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")
    target_url = f"{MASTERS_URL}/123456?token=test_user_token"
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_master_release(user=test_user, master_id=123456)

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_master_release___authentication_user_token()

In [ ]:
#export


def get_releases_related_to_master_release(user: Union[UserWithoutAuthentication,
                                                       UserWithUserTokenBasedAuthentication], 
                                           master_id: int,
                                           page: Union[int, None] = None,
                                           per_page: Union[int, None] = None,
                                           format: Union[str, None] = None,
                                           label: Union[str, None] = None,
                                           released: Union[str, None] = None, 
                                           country: Union[str, None] = None,
                                           sort: Union[str, None] = None, 
                                           sort_order: Union[str, None] = None) -> requests.models.Response:
    """
    Get a list of all Releases that are versions of a given master release.
    
    No user Authentication needed.
    """
    url = f"{MASTERS_URL}/{master_id}/versions"
    headers = user.headers
    params = user.params
    if page:
        params["page"] = max(1, page)
    if per_page:
        params["per_page"] = max(1, per_page)
    if format:
        params["format"] = format
    if label:
        params["label"] = label
    if released:
        params["released"] = released
    if country:
        params["country"] = country
    if sort and sort in VALID_SORT_OPTIONS:
        params["sort"] = sort
    if sort_order and sort_order in VALID_SORT_ORDER:
        params["sort_order"] = sort_order
    return requests.get(url, headers=headers, params=params)

In [ ]:
# Example for usage of get_master_release

get_releases_related_to_master_release(example_user, master_id=57960, 
                                       format="Cassette", label="Roadrunner Records")

<Response [200]>

In [ ]:
#hide

@responses.activate
def test_get_releases_related_to_master_release___authentication_none():
    test_user = UserWithoutAuthentication()
    target_url = f"{MASTERS_URL}/123456/versions?page=1&per_page=50&format=CD&label=label&" \
                 f"released=1989&country=country&sort=released&sort_order=asc"
    
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_releases_related_to_master_release(user=test_user, master_id=123456,
                                                  page=1, per_page=50, format="CD",
                                                  label="label", released="1989",
                                                  country="country", sort="released",
                                                  sort_order="asc")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url

test_get_releases_related_to_master_release___authentication_none()

@responses.activate
def test_get_releases_related_to_master_release___authentication_user_token():
    test_user = UserWithUserTokenBasedAuthentication(user_token="test_user_token",
                                                     user_agent="test_user_agent")

    target_url = f"{MASTERS_URL}/123456/versions?token=test_user_token&page=1&per_page=50&" \
                 f"format=CD&label=label&released=1989&country=country&sort=released&sort_order=asc"
    result_content = {"TestMaster": "TestContent"}
    
    responses.add(responses.GET, target_url,
                  json=result_content, status=200)

    resp = get_releases_related_to_master_release(user=test_user, master_id=123456,
                                                  page=1, per_page=50, format="CD",
                                                  label="label", released="1989",
                                                  country="country", sort="released",
                                                  sort_order="asc")

    assert resp.json() == result_content
    assert len(responses.calls) == 1
    assert responses.calls[0].request.url == target_url
    
test_get_releases_related_to_master_release___authentication_user_token()

In [ ]:
#hide

from nbdev.export import *
notebook2script()

Converted 00_core.constants.ipynb.
Converted 01_core.discogs_user.ipynb.
Converted 02_core.database_rq.ipynb.
Converted index.ipynb.
